In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from paths import add_paths_to_sys
add_paths_to_sys()

from datafold.pcfold.timeseries import TSCDataFrame
from datafold.dynfold.dmd import DMDFull

from limit_cycle import LimitCycle

# Set up mesh

In [ ]:
nr_time_steps = 30
dt = 0.01

x_nr_points = 11
y_nr_points = 11
mesh_x1, mesh_x2 = np.meshgrid(
    np.linspace(-2, 2, x_nr_points), np.linspace(-2, 2, y_nr_points)
)

# Sample time series from limit cycle

In [ ]:
tsc_data = None
for i in range(x_nr_points):
    for j in range(y_nr_points):
        generate_data = LimitCycle(eps=1, analytical=True).run(
            x1=mesh_x1[i, j], x2=mesh_x2[i, j], dt=dt, nr_steps=nr_time_steps
        )

        if tsc_data is None:
            tsc_data = TSCDataFrame.from_single_timeseries(generate_data)
        else:
            tsc_data = tsc_data.insert_ts(df=generate_data)

# Compute DMD model (dictionary is identity)

In [ ]:
tsc_dmd = tsc_data.copy(deep=True)
dmd = DMDFull().fit(X=tsc_dmd)
dmd_values = dmd.predict(tsc_dmd.initial_states_df())

# Compute E-DMD model (dictionary: add squared and cubed data)

In [ ]:
tsc_edmd = tsc_data.copy(deep=True)
tsc_edmd["radius_squared"] = np.square(tsc_edmd["radius"])
tsc_edmd["radius_cubed"] = np.power(tsc_edmd["radius"], 3)

In [ ]:
edmd = DMDFull().fit(X=tsc_edmd)  # Note that the "extended" part is in the transformations
edmd_values = edmd.predict(tsc_edmd.initial_states_df())

# Visually compare original data with DMD and EDMD model

In [ ]:
plt.figure(figsize=[15,9])
for _, df in tsc_data.itertimeseries():
    plt.polar(df["alpha"], df["radius"], c="b", linewidth=1)
plt.title("Original data")

plt.figure(figsize=[15,9])
for _, df in dmd_values.itertimeseries():
    plt.polar(df["alpha"], df["radius"], c="k", linewidth=1)
plt.title("DMD model evaluation")
    
plt.figure(figsize=[15,9])
for _, df in edmd_values.itertimeseries():
    plt.polar(df["alpha"], df["radius"], c="r", linewidth=1)
plt.title("EDMD model evaluation");
    